In [2]:
import pandas as pd
import numpy as np

# Домашняя работа

### 1. Провести расчет базовых метрик для них, кратко описать результаты.

* Опрерации - 180 лошадей перенесли операцию, 119 поправились без операции.
* Возраст - 276 из 300 взрослые особи. Остальные имеют индекс 9, что противоречит описанию (предполагаю что это ошибка).
* Темп-ра - только ц 25% лошадей температура соотв. норме (37,8%). У большей чати она выше нормы, что говорит о инфекции.
* Пульс - мнее 25% лошалей имеют нормальный пульс. Максимальное занчение 184 удара в мин, что скорее всего является ошибкой       измерения. Var = 819.71 - очень высокая.
* ЧастотаДых - Только у 6-ти лошадей в норме. У остальных превышена. Moda 20, что в 2 раза выше нормы.
* ТемпКонечн - 78-норма, 109-прохладные, 30 теплые, 27-холодные
* ПерифПульс - 115-норма, 5-увеличен, 104-уменьшен, 8-отсутсвует.

### 2. Работа с выбросами

* Нашел выбросы в возрасте.

### 3. Пропуски

* Пропуски по столбцу Температура заполнил, сгруппировав по переменной ТемпКонечн (т.к. в описании упоминалось о зависимости).    * Оставшиеся пропуски в Температуре и Операции заполнил медианой.
* Далее, по аналогичной схеме заполнил пропуски по остальным столбцам (группировка по переменной и остаток медианой).

In [3]:
# horse_data = pd.read_csv('horse_data.csv')
horse_data = pd.read_csv('horse_data.csv', header=None, na_values='?')
anslis_df = horse_data.loc[:, 0:7]
anslis_df.columns = ['Операция','Возраст', 'Клиника', 'Темп', 'Пульс', 'ЧастотаДых', 'ТемпКонечн', 'ПериферичПульс']
anslis_df


,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN
...,...,...,...,...,...,...,...,...
295,1.0,1,533886,NaN,120.0,70.0,4.0,NaN
296,2.0,1,527702,37.2,72.0,24.0,3.0,2.0
297,1.0,1,529386,37.5,72.0,30.0,4.0,3.0
298,1.0,1,530612,36.5,100.0,24.0,3.0,3.0


In [4]:
for col in anslis_df.columns:
    pct_missing = anslis_df[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

Операция - 0.3%
Возраст - 0.0%
Клиника - 0.0%
Темп - 20.0%
Пульс - 8.0%
ЧастотаДых - 19.3%
ТемпКонечн - 18.7%
ПериферичПульс - 23.0%


In [5]:
anslis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Операция        299 non-null    float64
 1   Возраст         300 non-null    int64  
 2   Клиника         300 non-null    int64  
 3   Темп            240 non-null    float64
 4   Пульс           276 non-null    float64
 5   ЧастотаДых      242 non-null    float64
 6   ТемпКонечн      244 non-null    float64
 7   ПериферичПульс  231 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 18.9 KB


In [6]:
anslis_df.describe()

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
count,299.000000,300.000000,3.000000e+02,240.000000,276.000000,242.000000,244.000000,231.000000
mean,1.397993,1.640000,1.085889e+06,38.167917,71.913043,30.417355,2.348361,2.017316
std,0.490305,2.173972,1.529801e+06,0.732289,28.630557,17.642231,1.045054,1.042428
min,1.000000,1.000000,5.184760e+05,35.400000,30.000000,8.000000,1.000000,1.000000
25%,1.000000,1.000000,5.289040e+05,37.800000,48.000000,18.500000,1.000000,1.000000
50%,1.000000,1.000000,5.303055e+05,38.200000,64.000000,24.500000,3.000000,2.000000
75%,2.000000,1.000000,5.347275e+05,38.500000,88.000000,36.000000,3.000000,3.000000
max,2.000000,9.000000,5.305629e+06,40.800000,184.000000,96.000000,4.000000,4.000000


In [7]:
print('Moda Операция:', anslis_df.Операция.mode()[0])
print('Var Операция:', anslis_df.Операция.var())

print('Moda Возраст:', anslis_df.Возраст.mode()[0])
print('Var Возраст:', anslis_df.Возраст.var())

print('Moda Темп:',anslis_df.Темп.mode()[0])
print('Var Темп:', anslis_df.Темп.var())

print('Moda Пульс:',anslis_df.Пульс.mode()[0])
print('Var Пульс:', anslis_df.Пульс.var())

print('Moda ЧастотаДых:',anslis_df.ЧастотаДых.mode()[0])
print('Var ЧастотаДых:', anslis_df.ЧастотаДых.var())

print('Moda ТемпКонечн:',anslis_df.ТемпКонечн.mode()[0])
print('Var ТемпКонечн:', anslis_df.ТемпКонечн.var())

print('Moda ПериферичПульс:',anslis_df.ПериферичПульс.mode()[0])
print('Var ПериферичПульс:', anslis_df.ПериферичПульс.var())

Moda Операция: 1.0
Var Операция: 0.24039864425040888
Moda Возраст: 1
Var Возраст: 4.726153846153814
Moda Темп: 38.0
Var Темп: 0.5362466875871686
Moda Пульс: 48.0
Var Пульс: 819.7087747035575
Moda ЧастотаДых: 20.0
Var ЧастотаДых: 311.24832824663054
Moda ТемпКонечн: 3.0
Var ТемпКонечн: 1.0921372191863987
Moda ПериферичПульс: 1.0
Var ПериферичПульс: 1.0866553736118991


In [8]:
anslis_df.Возраст.value_counts()

1    276
9     24
Name: Возраст, dtype: int64

In [9]:
q1 = anslis_df.Возраст.quantile(0.25)
q3 = anslis_df.Возраст.quantile(0.75)
#iqr = q3 - q1
lower_bound = q1
upper_bound = q3
remove_outliers = anslis_df[anslis_df.Возраст.between(lower_bound, upper_bound, inclusive=True)].sort_values('Возраст')
remove_outliers

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0
190,1.0,1,528742,37.1,40.0,8.0,NaN,1.0
192,1.0,1,534788,38.2,48.0,18.0,1.0,1.0
193,1.0,1,529373,NaN,60.0,48.0,3.0,3.0
...,...,...,...,...,...,...,...,...
100,1.0,1,535364,38.2,40.0,16.0,3.0,3.0
99,2.0,1,530002,39.6,108.0,51.0,3.0,3.0
98,2.0,1,534135,37.7,44.0,40.0,2.0,1.0
105,1.0,1,530381,NaN,70.0,36.0,1.0,NaN


In [10]:
remove_outliers.describe()

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
count,275.000000,276.0,2.760000e+02,220.000000,254.000000,222.000000,228.000000,217.000000
mean,1.410909,1.0,7.717916e+05,38.124545,67.456693,28.238739,2.364035,2.009217
std,0.492896,0.0,1.044374e+06,0.735988,23.933953,15.251192,1.042669,1.045231
min,1.000000,1.0,5.184760e+05,35.400000,30.000000,8.000000,1.000000,1.000000
25%,1.000000,1.0,5.288030e+05,37.700000,48.000000,18.000000,1.000000,1.000000
50%,1.000000,1.0,5.301290e+05,38.100000,60.000000,24.000000,3.000000,1.000000
75%,2.000000,1.0,5.342178e+05,38.500000,84.000000,36.000000,3.000000,3.000000
max,2.000000,1.0,5.299603e+06,40.800000,140.000000,96.000000,4.000000,4.000000


In [11]:
remove_outliers.Возраст.value_counts()

1    276
Name: Возраст, dtype: int64

In [12]:
for col in remove_outliers.columns:
    pct_missing = remove_outliers[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

Операция - 0.4%
Возраст - 0.0%
Клиника - 0.0%
Темп - 20.3%
Пульс - 8.0%
ЧастотаДых - 19.6%
ТемпКонечн - 17.4%
ПериферичПульс - 21.4%


In [13]:
remove_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Операция        275 non-null    float64
 1   Возраст         276 non-null    int64  
 2   Клиника         276 non-null    int64  
 3   Темп            220 non-null    float64
 4   Пульс           254 non-null    float64
 5   ЧастотаДых      222 non-null    float64
 6   ТемпКонечн      228 non-null    float64
 7   ПериферичПульс  217 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 19.4 KB


In [14]:
#print(remove_outliers.groupby(['ЧастотаДых', 'ТемпКонечн']).Темп.median())
# print(remove_outliers.groupby(['ТемпКонечн']).Темп.median())
#fill_median_by_temp = pd.read_csv('horse_data.csv', header=None, na_values='?')

remove_outliers.Темп.fillna(remove_outliers.groupby(['ТемпКонечн']).Темп.transform('median'), inplace=True)
remove_outliers


,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0
190,1.0,1,528742,37.1,40.0,8.0,NaN,1.0
192,1.0,1,534788,38.2,48.0,18.0,1.0,1.0
193,1.0,1,529373,38.1,60.0,48.0,3.0,3.0
...,...,...,...,...,...,...,...,...
100,1.0,1,535364,38.2,40.0,16.0,3.0,3.0
99,2.0,1,530002,39.6,108.0,51.0,3.0,3.0
98,2.0,1,534135,37.7,44.0,40.0,2.0,1.0
105,1.0,1,530381,38.1,70.0,36.0,1.0,NaN


In [15]:
remove_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Операция        275 non-null    float64
 1   Возраст         276 non-null    int64  
 2   Клиника         276 non-null    int64  
 3   Темп            264 non-null    float64
 4   Пульс           254 non-null    float64
 5   ЧастотаДых      222 non-null    float64
 6   ТемпКонечн      228 non-null    float64
 7   ПериферичПульс  217 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 19.4 KB


In [16]:
remove_outliers.Темп.fillna(remove_outliers.Темп.median(), inplace=True)
remove_outliers.Операция.fillna(remove_outliers.Операция.median(), inplace=True)

remove_outliers

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0
190,1.0,1,528742,37.1,40.0,8.0,NaN,1.0
192,1.0,1,534788,38.2,48.0,18.0,1.0,1.0
193,1.0,1,529373,38.1,60.0,48.0,3.0,3.0
...,...,...,...,...,...,...,...,...
100,1.0,1,535364,38.2,40.0,16.0,3.0,3.0
99,2.0,1,530002,39.6,108.0,51.0,3.0,3.0
98,2.0,1,534135,37.7,44.0,40.0,2.0,1.0
105,1.0,1,530381,38.1,70.0,36.0,1.0,NaN


In [17]:
remove_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Операция        276 non-null    float64
 1   Возраст         276 non-null    int64  
 2   Клиника         276 non-null    int64  
 3   Темп            276 non-null    float64
 4   Пульс           254 non-null    float64
 5   ЧастотаДых      222 non-null    float64
 6   ТемпКонечн      228 non-null    float64
 7   ПериферичПульс  217 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 19.4 KB


In [18]:
#print(remove_outliers.groupby(['Темп']).ТемпКонечн.median())
remove_outliers.ТемпКонечн.fillna(remove_outliers.groupby('Темп').ТемпКонечн.transform('median'), inplace=True)
remove_outliers.ТемпКонечн.fillna(remove_outliers.ТемпКонечн.median(), inplace=True)
remove_outliers

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0
190,1.0,1,528742,37.1,40.0,8.0,3.0,1.0
192,1.0,1,534788,38.2,48.0,18.0,1.0,1.0
193,1.0,1,529373,38.1,60.0,48.0,3.0,3.0
...,...,...,...,...,...,...,...,...
100,1.0,1,535364,38.2,40.0,16.0,3.0,3.0
99,2.0,1,530002,39.6,108.0,51.0,3.0,3.0
98,2.0,1,534135,37.7,44.0,40.0,2.0,1.0
105,1.0,1,530381,38.1,70.0,36.0,1.0,NaN


In [19]:
remove_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Операция        276 non-null    float64
 1   Возраст         276 non-null    int64  
 2   Клиника         276 non-null    int64  
 3   Темп            276 non-null    float64
 4   Пульс           254 non-null    float64
 5   ЧастотаДых      222 non-null    float64
 6   ТемпКонечн      276 non-null    float64
 7   ПериферичПульс  217 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 19.4 KB


In [20]:
remove_outliers.ЧастотаДых.fillna(remove_outliers.groupby('Пульс').ЧастотаДых.transform('median'), inplace=True)

remove_outliers.ЧастотаДых.fillna(remove_outliers.ЧастотаДых.median(), inplace=True)

remove_outliers.Пульс.fillna(remove_outliers.Пульс.median(), inplace=True)

remove_outliers

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0
190,1.0,1,528742,37.1,40.0,8.0,3.0,1.0
192,1.0,1,534788,38.2,48.0,18.0,1.0,1.0
193,1.0,1,529373,38.1,60.0,48.0,3.0,3.0
...,...,...,...,...,...,...,...,...
100,1.0,1,535364,38.2,40.0,16.0,3.0,3.0
99,2.0,1,530002,39.6,108.0,51.0,3.0,3.0
98,2.0,1,534135,37.7,44.0,40.0,2.0,1.0
105,1.0,1,530381,38.1,70.0,36.0,1.0,NaN


In [21]:
remove_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Операция        276 non-null    float64
 1   Возраст         276 non-null    int64  
 2   Клиника         276 non-null    int64  
 3   Темп            276 non-null    float64
 4   Пульс           276 non-null    float64
 5   ЧастотаДых      276 non-null    float64
 6   ТемпКонечн      276 non-null    float64
 7   ПериферичПульс  217 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 19.4 KB


In [22]:
remove_outliers.ПериферичПульс.fillna(remove_outliers.groupby('Пульс').ПериферичПульс.transform('median'), inplace=True)

remove_outliers.ПериферичПульс.fillna(remove_outliers.ПериферичПульс.median(), inplace=True)

remove_outliers

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0
190,1.0,1,528742,37.1,40.0,8.0,3.0,1.0
192,1.0,1,534788,38.2,48.0,18.0,1.0,1.0
193,1.0,1,529373,38.1,60.0,48.0,3.0,3.0
...,...,...,...,...,...,...,...,...
100,1.0,1,535364,38.2,40.0,16.0,3.0,3.0
99,2.0,1,530002,39.6,108.0,51.0,3.0,3.0
98,2.0,1,534135,37.7,44.0,40.0,2.0,1.0
105,1.0,1,530381,38.1,70.0,36.0,1.0,4.0


In [23]:
remove_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Операция        276 non-null    float64
 1   Возраст         276 non-null    int64  
 2   Клиника         276 non-null    int64  
 3   Темп            276 non-null    float64
 4   Пульс           276 non-null    float64
 5   ЧастотаДых      276 non-null    float64
 6   ТемпКонечн      276 non-null    float64
 7   ПериферичПульс  276 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 19.4 KB


In [24]:
remove_outliers.describe()

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
count,276.00000,276.0,2.760000e+02,276.000000,276.000000,276.000000,276.000000,276.000000
mean,1.40942,1.0,7.717916e+05,38.125725,66.862319,28.067029,2.393116,1.974638
std,0.49262,0.0,1.044374e+06,0.657509,23.045628,14.011425,1.012821,1.006926
min,1.00000,1.0,5.184760e+05,35.400000,30.000000,8.000000,1.000000,1.000000
25%,1.00000,1.0,5.288030e+05,37.800000,48.000000,20.000000,1.000000,1.000000
50%,1.00000,1.0,5.301290e+05,38.100000,60.000000,24.000000,3.000000,2.000000
75%,2.00000,1.0,5.342178e+05,38.400000,80.500000,32.000000,3.000000,3.000000
max,2.00000,1.0,5.299603e+06,40.800000,140.000000,96.000000,4.000000,4.000000


In [26]:
# разница стат.показателей 
x = anslis_df.describe() - remove_outliers.describe()
x

,Операция,Возраст,Клиника,Темп,Пульс,ЧастотаДых,ТемпКонечн,ПериферичПульс
count,23.000000,24.000000,24.000000,-36.000000,0.000000,-34.000000,-32.000000,-45.000000
mean,-0.011427,0.640000,314097.188406,0.042192,5.050725,2.350326,-0.044755,0.042678
std,-0.002315,2.173972,485426.445830,0.074780,5.584929,3.630806,0.032232,0.035502
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,101.000000,0.000000,0.000000,-1.500000,0.000000,0.000000
50%,0.000000,0.000000,176.500000,0.100000,4.000000,0.500000,0.000000,0.000000
75%,0.000000,0.000000,509.750000,0.100000,7.500000,4.000000,0.000000,0.000000
max,0.000000,8.000000,6026.000000,0.000000,44.000000,0.000000,0.000000,0.000000
